<a href="https://colab.research.google.com/github/malowana/NLP_budzet_obywatelski/blob/main/BO_analiza_morfologiczna_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyLDAvis

In [15]:
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', 'SelectableGroups dict interface')

In [4]:
import pandas as pd
import numpy as np
import gensim
from gensim import corpora, models
from gensim.utils import simple_preprocess
import pyLDAvis.gensim_models as vis
import pyLDAvis

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [5]:
# Wyświetlanie szerszych kolumn, aby nie skracać wyświetlanych nazw tematów
pd.set_option('max_colwidth', 400)

## Warszawa

In [6]:
df_waw = pd.read_csv('/content/drive/My Drive/Konkursy Kaggle/Budżet obywatelski/Morfeusz_WAW.csv')
df_waw.head()

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_
0,2018,WARSZAWA,Wolskie lato filmowe,0,1,Wolskie,wolski,adj:pl:acc:m2.m3.f.n:pos,[],[]
1,2018,WARSZAWA,Wolskie lato filmowe,0,1,Wolskie,wolski,adj:pl:nom.voc:m2.m3.f.n:pos,[],[]
2,2018,WARSZAWA,Wolskie lato filmowe,0,1,Wolskie,wolski,adj:sg:acc:n:pos,[],[]
3,2018,WARSZAWA,Wolskie lato filmowe,0,1,Wolskie,wolski,adj:sg:nom.voc:n:pos,[],[]
4,2018,WARSZAWA,Wolskie lato filmowe,0,1,Wolskie,Wolskie,subst:pl:nom.acc.voc:n:pt,['nazwa_geograficzna'],[]


In [7]:
df_waw['lemma_clean'] = df_waw['lemma'].str.split(':', expand=True)[0]
df_waw.sample(20)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_,lemma_clean
31420,2018,WARSZAWA,Rewitalizacja nawierzchni na ul. Różanej na odcinku od Al Niepodległości do ul. Kazimierzowskiej,1,2,nawierzchni,nawierzchnia,subst:pl:gen:f,['nazwa_pospolita'],[],nawierzchnia
18608,2018,WARSZAWA,Sąsiedzkie gry planszowe,0,1,Sąsiedzkie,sąsiedzki,adj:pl:nom.voc:m2.m3.f.n:pos,[],[],sąsiedzki
41223,2019,WARSZAWA,Pierwsze parki kieszonkowe w Dzielnicy Włochy - tak dla zieleni !,0,1,Pierwsze,pierwsze,subst:pl:nom.acc.voc:n:ncol,['nazwa_pospolita'],[],pierwsze
29740,2018,WARSZAWA,ŻEGLARSKIE MARZENIE - otwarty plac zabaw przy ul. Limanowskiego,0,1,ŻEGLARSKIE,żeglarski,adj:pl:acc:m2.m3.f.n:pos,[],[],żeglarski
85676,2020,WARSZAWA,"Zdrowo i na sportowo - cały rok zajęcia sportowe dla mieszkańców dzielnicy: callanetics, zdrowy kręgosłup, pilates, joga, gimnastyka dla seniorów, aerobic, boks, aqua fitness, nordic walking",13,14,callanetics,callanetics,subst:sg:nom.acc:m3,['nazwa_pospolita'],[],callanetics
19512,2018,WARSZAWA,Biblioteka przyjazna dla osób z dysfunkcją wzroku,4,5,z,z:P,prep:gen:nwok,[],[],z
21080,2018,WARSZAWA,Wszelkie wózki i rowery na Burakowską,5,6,Burakowską,Burakowska,subst:sg:inst:f,['nazwisko'],[],Burakowska
53922,2019,WARSZAWA,Zabawy i techniki relaksacyjne dla dzieci i młodzieży (cykl zajęć dla dzieci w szkołach i przedszkolach),9,10,cykl,cykl,subst:sg:nom.acc:m3,['nazwa_pospolita'],[],cykl
30753,2018,WARSZAWA,Czyste Psie Nadwiśle,0,1,Czyste,czysty,adj:pl:acc:m2.m3.f.n:pos,[],[],czysty
5741,2018,WARSZAWA,Poprawa bezpieczeństwa pieszych w rejonie ul. Świetlików,0,1,Poprawa,Poprawa:Sm1,subst:sg:nom:m1,['nazwisko'],[],Poprawa


In [8]:
df_waw['tokens'] = df_waw['lemma_clean'].apply(simple_preprocess)
df_waw.sample(20)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_,lemma_clean,tokens
5521,2018,WARSZAWA,Bezpieczne przejścia dla pieszych,3,4,pieszych,pieszy:A,adj:pl:loc:m1.m2.m3.f.n:pos,[],[],pieszy,[pieszy]
43134,2019,WARSZAWA,Psie pole,0,1,Psie,pies:Sm2,subst:sg:loc:m2,['nazwa_pospolita'],[],pies,[pies]
74612,2020,WARSZAWA,"Chronimy ptaki, wiewiórki, nietoperze i owady zapylające na Pradze Północ-konserwacja i zawieszanie skrzynek lęgowych, zimowe dokarmianie ptaków, utrzymanie schronień dla owadów",9,10,na,na:I,interj,[],[],na,[na]
48772,2019,WARSZAWA,Barierki ochronne wzdłuż chodników po zachodniej stronie ulicy Jana Pawła II odcinku od ul. Trakt Brzeski do ul. Gościniec,10,11,II,II,romandig,[],[],II,[ii]
47947,2019,WARSZAWA,"Wymiana nawierzchni jezdni na ulicy Baltazara, na odcinku Szulborska - Nakielska oraz poprawa bezpieczeństwa.",5,6,Baltazara,Baltazar,subst:sg:gen.acc:m1,['imię'],[],Baltazar,[baltazar]
513,2018,WARSZAWA,"Dofinansowanie zakupu książek, audiobooków i literatury obcojęzycznej dla Biblioteki Publicznej w dzielnicy Bielany",0,1,Dofinansowanie,dofinansowanie,subst:sg:nom.acc.voc:n:ncol,['nazwa_pospolita'],[],dofinansowanie,[dofinansowanie]
34868,2019,WARSZAWA,"Naturalny, przyjazny przyrodzie trawnik w parku Szczęśliwice",4,5,trawnik,trawnik,subst:sg:nom.acc:m3,['nazwa_pospolita'],[],trawnik,[trawnik]
71893,2020,WARSZAWA,Stare płyty do lamusa - remont chodnika na Mołdawskiej,0,1,Stare,stary:A,adj:sg:acc:n:pos,[],[],stary,[stary]
21259,2018,WARSZAWA,W olimpijskim blasku - spotkania uczniów szkół podstawowych z utytułowanymi sportowcami,8,9,z,z:P,prep:gen:nwok,[],[],z,[]
88176,2020,WARSZAWA,Nawadnianie drzew - treegatory - Żoliborz Artystyczny i Żoliborz Południowy,0,1,Nawadnianie,nawadniać,ger:sg:nom.acc:n:imperf:aff,[],[],nawadniać,[nawadniać]


In [9]:
df_waw3 = df_waw[df_waw["tokens"].str.len() != 0]
df_waw3.sample(60)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_,lemma_clean,tokens
1533,2018,WARSZAWA,Sto drzew dla Wilanowa,2,3,dla,dla,prep:gen,[],[],dla,[dla]
59264,2019,WARSZAWA,Nowy chodnik i drzewa wyciszające na granicy Ursynowa oraz Mokotowa,4,5,wyciszające,wyciszać,pact:sg:nom.acc.voc:n:imperf:aff,[],[],wyciszać,[wyciszać]
27537,2018,WARSZAWA,Zrozumieć psa - cykl spotkań z behawiorystą/trenerem w Psim Parku,10,11,Psim,psi:A,adj:pl:dat:m1.m2.m3.f.n:pos,[],[],psi,[psi]
53640,2019,WARSZAWA,"Rembertowska Warsztatownia, czyli cykl praktycznych warsztatów dla każdego",7,8,dla,dla,prep:gen,[],[],dla,[dla]
24948,2018,WARSZAWA,"Spotkania w Parku Sady Żoliborskie. (Kultura, Geografia, Sport, Środowisko)",13,14,Środowisko,środowisko,subst:sg:nom.acc.voc:n:ncol,['nazwa_pospolita'],[],środowisko,[środowisko]
66128,2020,WARSZAWA,Ścieżka zdrowia z urządzeniami do kalisteniki (street workout) w bielańskiej części Lasu Bemowskiego.,0,1,Ścieżka,ścieżek,subst:sg:gen:m3,['nazwa_pospolita'],[],ścieżek,[ścieżek]
13463,2018,WARSZAWA,Bezpieczna droga do szkoły nr 257 nowy chodnik i przejście dla pieszych ul. Żywiczna,6,7,nowy,nowy:S,subst:sg:voc:m1,['nazwa_pospolita'],[],nowy,[nowy]
66878,2020,WARSZAWA,Modernizacja placu zabaw,1,2,placu,plac,subst:sg:gen:m3,['nazwa_pospolita'],[],plac,[plac]
18615,2018,WARSZAWA,Sąsiedzkie gry planszowe,2,3,planszowe,planszowy,adj:sg:acc:n:pos,[],[],planszowy,[planszowy]
76163,2020,WARSZAWA,Bezpieczna droga do szkoły podstawowej 217 - doświetlenie przejść dla pieszych przy ul. Paderewskiego i wyznaczenie nowego przejścia Kordiana-Paderewskiego,10,11,pieszych,pieszy:A,adj:pl:acc:m1:pos,[],[],pieszy,[pieszy]


In [10]:
processed_docs = df_waw3['tokens']

In [11]:
vocab = gensim.corpora.Dictionary(processed_docs)
len(vocab)

5958

In [12]:
bow_corpus = [vocab.doc2bow(doc) for doc in processed_docs]
bow_corpus[10]

[(4, 1)]

In [16]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=vocab, passes=2)

In [17]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
    print("")

Topic: 0 
Words: 0.064*"przy" + 0.051*"podstawowy" + 0.034*"woli" + 0.031*"drzewo" + 0.028*"co" + 0.025*"muzyczny" + 0.020*"ursynów" + 0.019*"remonta" + 0.018*"wolski" + 0.017*"radość"

Topic: 1 
Words: 0.078*"pieszy" + 0.070*"do" + 0.038*"warsztat" + 0.034*"rowerowy" + 0.031*"kultura" + 0.030*"zabawa" + 0.024*"lęgowy" + 0.022*"mały" + 0.021*"dorosła" + 0.021*"przyjazny"

Topic: 2 
Words: 0.034*"plenerowy" + 0.031*"wawerski" + 0.029*"on" + 0.024*"stary" + 0.021*"to" + 0.018*"zielone" + 0.018*"wysokość" + 0.016*"nowe" + 0.015*"siłownia" + 0.013*"kępa"

Topic: 3 
Words: 0.096*"dziecko" + 0.079*"ul" + 0.047*"mini" + 0.046*"zajęcie" + 0.037*"warsztaty" + 0.023*"spotkać" + 0.019*"remont" + 0.019*"nawierzchnia" + 0.018*"nie" + 0.015*"aktywny"

Topic: 4 
Words: 0.140*"dla" + 0.118*"wola" + 0.060*"szkoła" + 0.060*"park" + 0.041*"zająć" + 0.039*"nowa" + 0.024*"wzdłuż" + 0.021*"ursus" + 0.012*"nowość" + 0.012*"wszyscy"

Topic: 5 
Words: 0.071*"plac" + 0.068*"warszawa" + 0.064*"zieleń" + 0.054*"d

In [18]:
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [19]:
lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=5, id2word=vocab, passes=2, workers=10)

In [20]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.075*"dla" + 0.031*"młodzież" + 0.027*"na" + 0.023*"rowerowy" + 0.022*"bezpłatny" + 0.021*"zabawa" + 0.018*"chodnik" + 0.012*"edukacyjny" + 0.011*"numer" + 0.011*"zajęcia"
Topic: 1 
Words: 0.049*"nowy" + 0.048*"dziecko" + 0.036*"warszawa" + 0.029*"dorosły" + 0.024*"wola" + 0.019*"nowa" + 0.015*"drzewo" + 0.014*"on" + 0.014*"mały" + 0.012*"spotkać"
Topic: 2 
Words: 0.217*"na" + 0.054*"zielony" + 0.026*"szkoła" + 0.021*"zająć" + 0.014*"plenerowy" + 0.014*"stary" + 0.012*"nowość" + 0.012*"wzdłuż" + 0.011*"zabawić" + 0.010*"po"
Topic: 3 
Words: 0.050*"ulica" + 0.031*"park" + 0.024*"mini" + 0.016*"zajęcie" + 0.015*"praga" + 0.014*"dzielnica" + 0.014*"podstawowy" + 0.013*"warsztaty" + 0.009*"nr" + 0.009*"sportowy"
Topic: 4 
Words: 0.040*"do" + 0.036*"ul" + 0.035*"pieszy" + 0.033*"przy" + 0.032*"plac" + 0.026*"zieleń" + 0.020*"bezpieczny" + 0.017*"biblioteka" + 0.016*"warsztat" + 0.014*"oraz"


In [21]:
tags = ['prep', 'conj', 'interj', 'part']
df_waw_clean = df_waw3[~df_waw3['tag'].str.contains('|'.join(tags))]
df_waw_clean.sample(60)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_,lemma_clean,tokens
12212,2018,WARSZAWA,Łąka na bulwarach - EKOMIASTO,4,5,EKOMIASTO,ekoMiasto,subst:sg:nom.acc.voc:n:ncol,['człon_nazwy_geograficznej'],[],ekoMiasto,[ekomiasto]
52582,2019,WARSZAWA,Kurs komputerowy dla seniorów edycja II,3,4,seniorów,senior,subst:pl:gen.acc:m1,['nazwa_pospolita'],[],senior,[senior]
3093,2018,WARSZAWA,remont chodnika przy ul. Belgradzkiej między ul. Rosoła a Nowoursynowską,1,2,chodnika,chodnik,subst:sg:gen:m3,['nazwa_pospolita'],[],chodnik,[chodnik]
32740,2019,WARSZAWA,Modernizacja drogi rowerowej wzdłuż ul. Belgradzkiej na odcinku od al. KEN do ul. Rosoła,1,2,drogi,drogi:S,subst:pl:nom.acc.voc:n:pt,['nazwa_pospolita'],[],drogi,[drogi]
76168,2020,WARSZAWA,Bezpieczna droga do szkoły podstawowej 217 - doświetlenie przejść dla pieszych przy ul. Paderewskiego i wyznaczenie nowego przejścia Kordiana-Paderewskiego,12,13,ul,ul,subst:sg:nom.acc:m3,['nazwa_pospolita'],[],ul,[ul]
75458,2020,WARSZAWA,Bezpieczna droga do szkoły SP 217 - doświetlenie przejścia dla pieszych Paderewskiego-Kaletnicza i przesunięcie przystanku Szafarzy 02,1,2,droga,droga,subst:sg:nom:f,['nazwa_pospolita'],[],droga,[droga]
77034,2020,WARSZAWA,Potańcówka na Mariensztacie,2,3,Mariensztacie,Mariensztat,subst:sg:loc:m3,['nazwa_geograficzna'],[],Mariensztat,[mariensztat]
15301,2018,WARSZAWA,"Zakup sprzętu AUDIO - WIDEO dla Domu Kultury ""Kolorowa""",4,5,WIDEO,wideo:A,adj:pl:acc:m2.m3.f.n:pos,[],[],wideo,[wideo]
28327,2018,WARSZAWA,Nowy skwer miejski we Włochach. Potrzebna róg Plastycznej.,6,7,Potrzebna,potrzebny:A,adj:sg:nom.voc:f:pos,[],[],potrzebny,[potrzebny]
31353,2018,WARSZAWA,Przejście dla pieszych przy przystanku Barszczewska,2,3,pieszych,pieszy:A,adj:pl:acc:m1:pos,[],[],pieszy,[pieszy]


In [22]:
processed_docs = df_waw_clean['tokens']
vocab = gensim.corpora.Dictionary(processed_docs)
bow_corpus = [vocab.doc2bow(doc) for doc in processed_docs]

lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=5, id2word=vocab, passes=2)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
    print("")

Topic: 0 
Words: 0.069*"ulica" + 0.037*"warszawa" + 0.037*"plac" + 0.020*"sport" + 0.020*"plenerowy" + 0.019*"warsztaty" + 0.014*"lęgowy" + 0.013*"świąteczny" + 0.013*"rodzinny" + 0.013*"spotkać"

Topic: 1 
Words: 0.058*"wola" + 0.027*"dorosły" + 0.018*"do" + 0.015*"kultura" + 0.015*"zabawa" + 0.015*"muzyczny" + 0.014*"boisko" + 0.012*"zajęcia" + 0.011*"przasnyski" + 0.011*"dorosła"

Topic: 2 
Words: 0.051*"nowy" + 0.038*"pieszy" + 0.035*"szkoła" + 0.031*"młodzież" + 0.028*"podstawowy" + 0.023*"mini" + 0.023*"zająć" + 0.023*"zajęcie" + 0.023*"bezpieczny" + 0.022*"biblioteka"

Topic: 3 
Words: 0.068*"zielony" + 0.056*"dziecko" + 0.039*"zieleń" + 0.028*"nowa" + 0.018*"praga" + 0.013*"sportowy" + 0.012*"zabawić" + 0.012*"remonta" + 0.012*"zielona" + 0.012*"mały"

Topic: 4 
Words: 0.044*"ul" + 0.040*"park" + 0.022*"dzielnica" + 0.022*"żoliborski" + 0.021*"chodnik" + 0.019*"bezpłatny" + 0.019*"warsztat" + 0.018*"rowerowy" + 0.016*"sady" + 0.012*"dom"



In [23]:
#tfidf
processed_docs = df_waw_clean['tokens']
vocab = gensim.corpora.Dictionary(processed_docs)
corpus = [vocab.doc2bow(doc) for doc in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=5, id2word=vocab, passes=2, workers=10)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.059*"nowy" + 0.058*"ulica" + 0.055*"wola" + 0.039*"pieszy" + 0.037*"plac" + 0.024*"nowa" + 0.021*"bezpłatny" + 0.017*"stary" + 0.016*"woli" + 0.013*"książka"
Topic: 1 
Words: 0.038*"ul" + 0.027*"zajęcie" + 0.027*"do" + 0.021*"chodnik" + 0.018*"warsztaty" + 0.016*"nowość" + 0.015*"kultura" + 0.015*"mały" + 0.014*"zabawić" + 0.012*"przyjazny"
Topic: 2 
Words: 0.026*"młodzież" + 0.024*"mini" + 0.022*"rowerowy" + 0.014*"wawerski" + 0.012*"sportowy" + 0.011*"sport" + 0.009*"zielone" + 0.009*"pies" + 0.008*"to" + 0.008*"wypożyczalnia"
Topic: 3 
Words: 0.067*"zielony" + 0.051*"dziecko" + 0.032*"warszawa" + 0.027*"zieleń" + 0.026*"dorosły" + 0.024*"biblioteka" + 0.022*"podstawowy" + 0.018*"plenerowy" + 0.017*"warsztat" + 0.015*"praga"
Topic: 4 
Words: 0.046*"park" + 0.031*"szkoła" + 0.025*"zająć" + 0.021*"dzielnica" + 0.020*"bezpieczny" + 0.020*"zabawa" + 0.014*"on" + 0.014*"numer" + 0.013*"drzewo" + 0.012*"nr"


In [24]:
# Wizualizacja tematów
vis_data = vis.prepare(lda_model, corpus_tfidf, vocab)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis_data)

In [25]:
stopwords = ['do', 'wraz', 'oraz', 'ul', 'ulica', 'nr', 'numer', 'to', 'on', 'przy', 'dla', 'na', 'po','jak', 'ten', 'nie', 'od', 'oda', 'czyli']
df_waw_clean4 = df_waw3[~df_waw3['lemma'].str.contains('|'.join(stopwords))]
df_waw_clean4.sample(10)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_,lemma_clean,tokens
65421,2020,WARSZAWA,"Pieniążki na filmy i książki - wspieramy białołęckie biblioteki: Antalla, Berensona, Pancera, Wałuszewska, Van Gogha",0,1,Pieniążki,pieniążek:Sm2,subst:pl:nom.acc.voc:m2,['nazwa_pospolita'],['mikol.'],pieniążek,[pieniążek]
43107,2019,WARSZAWA,Bezpłatne zajęcia/spotkania dla Seniorów w DK Kadr.,0,1,Bezpłatne,bezpłatny,adj:sg:acc:n:pos,[],[],bezpłatny,[bezpłatny]
11255,2018,WARSZAWA,"Pij, Warszawo - na Grochowie Północnym",6,7,Północnym,północny,adj:sg:loc:m1.m2.m3.n:pos,[],[],północny,[północny]
8699,2018,WARSZAWA,"Aleja Stefanika - wzdłuż ul. Światowida, przy osiedlu Stefanika",9,10,osiedlu,osiedle,subst:sg:dat:n:ncol,['nazwa_pospolita'],[],osiedle,[osiedle]
1607,2018,WARSZAWA,Alejka i zielony skwerek przy Parku Magiczna,5,6,Parku,park,subst:sg:gen:m3,['nazwa_pospolita'],[],park,[park]
32603,2019,WARSZAWA,"Lazurowa Wyspa - ogólnodostępny plac zabaw dla dzieci w wieku od 1 roku życia z elementami sensorycznymi i integracyjnymi, położony w Parku Górczewska w pobliżu ul. Lazurowej",13,14,życia,życie,subst:pl:nom.acc.voc:n:ncol,['nazwa_pospolita'],[],życie,[życie]
15183,2018,WARSZAWA,Zakup nowości książkowych do Wypożyczalni nr 40 i Biblioteki dla Dzieci nr 47,2,3,książkowych,książkowy,adj:pl:loc:m1.m2.m3.f.n:pos,[],[],książkowy,[książkowy]
73253,2020,WARSZAWA,Siłownia w Prusie,2,3,Prusie,Prus:Sm1,subst:sg:loc:m1,"['nazwa_pospolita', 'nazwisko']",[],Prus,[prus]
52166,2019,WARSZAWA,Monitoring wizyjny w Marysinie Północnym,4,5,Północnym,północny,adj:sg:loc:m1.m2.m3.n:pos,[],[],północny,[północny]
51891,2019,WARSZAWA,Bieg po Targówku,0,1,Bieg,bieg,subst:sg:nom.acc:m3,['nazwa_pospolita'],[],bieg,[bieg]


In [26]:
#tfidf
processed_docs = df_waw_clean4['tokens']
vocab = gensim.corpora.Dictionary(processed_docs)
corpus = [vocab.doc2bow(doc) for doc in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=6, id2word=vocab, passes=2, workers=10)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.081*"dziecko" + 0.046*"mini" + 0.045*"szkoła" + 0.033*"dzielnica" + 0.028*"bezpieczny" + 0.022*"warsztaty" + 0.019*"drzewo" + 0.017*"boisko" + 0.017*"skwer" + 0.016*"angielski"
Topic: 1 
Words: 0.078*"park" + 0.025*"warsztat" + 0.023*"wzdłuż" + 0.017*"edukacyjny" + 0.017*"centrum" + 0.017*"gra" + 0.016*"wawerski" + 0.015*"pies" + 0.015*"oświetlić" + 0.014*"nowe"
Topic: 2 
Words: 0.047*"wola" + 0.043*"warszawa" + 0.033*"nowa" + 0.027*"zająć" + 0.017*"mały" + 0.016*"ursus" + 0.012*"wszystek" + 0.011*"ochota" + 0.010*"zakupić" + 0.009*"targówek"
Topic: 3 
Words: 0.028*"plenerowy" + 0.022*"zajęcia" + 0.018*"warszawski" + 0.017*"miejsce" + 0.017*"przejście" + 0.016*"senior" + 0.015*"publiczny" + 0.015*"ścieżka" + 0.015*"woli" + 0.014*"miejski"
Topic: 4 
Words: 0.072*"pieszy" + 0.035*"rowerowy" + 0.035*"biblioteka" + 0.028*"bezpłatny" + 0.028*"praga" + 0.022*"zabawić" + 0.022*"stary" + 0.018*"nowość" + 0.017*"ursynów" + 0.014*"ławka"
Topic: 5 
Words: 0.106*"nowy" + 0.048*"

## Reszta miast

In [27]:
df_roc = pd.read_csv('/content/drive/My Drive/Konkursy Kaggle/Budżet obywatelski/Morfeusz_ROC.csv')
df_roc.head()

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_
0,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",0,1,Doposażenie,doposażyć,ger:sg:nom.acc:n:perf:aff,[],[]
1,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",1,2,parku,park,subst:sg:gen:m3,['nazwa_pospolita'],[]
2,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",1,2,parku,park,subst:sg:loc:m3,['nazwa_pospolita'],[]
3,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",1,2,parku,park,subst:sg:voc:m3,['nazwa_pospolita'],[]
4,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",2,3,pomiędzy,pomiędzy,prep:acc,[],[]


In [28]:
df_roc['lemma_clean'] = df_roc['lemma'].str.split(':', expand=True)[0]
df_roc.sample(20)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_,lemma_clean
15454,2019,GDYNIA,"Wynajem przestrzeni na biblioteke, czytelniq i miejsce spotkan mieszkanców, zakup wyposazenia, warsztaty pracowni brudnej W Pustkach Cisowskich-Dempotowie i wydawanie gazetki",24,25,gazetki,gazetka,subst:pl:nom.acc.voc:f,['nazwa_pospolita'],[],gazetka
156983,2020,KIELCE,"""Dwójka Gola ! - Boisko jest dobre na wszystko""",7,8,dobre,dobre,subst:pl:nom.acc.voc:n:ncol,['nazwa_pospolita'],[],dobre
168803,2019,OLSZTYN,157. REMONT NAWIERZCHNI CHODNIKA PRZY UL. GĘBIKA (PO STRONIE BUDYNKÓW O NUMERACJI NIEPARZYSTEJ).,12,13,BUDYNKÓW,Budynek:Sm1,subst:pl:gen.acc:m1,['nazwisko'],[],Budynek
181159,2020,KONIN,WIATA ROWEROWA NA V OSIEDLU wniosek,0,1,WIATA,wiata,subst:sg:nom:f,['nazwa_pospolita'],[],wiata
377732,2019,KRAKÓW,Mobilne lodowisko w Czyżynach!,0,1,Mobilne,mobilny,adj:sg:nom.voc:n:pos,[],[],mobilny
56307,2019,BIELSKO-BIAŁA,"Wykonanie miejsc parkingowych, w tym dla osób z małymi dziećmi oraz niepełnosprawnych, obok pawilonu handlowo-usługowego przy ul. Stawowej 29 w Bielsku-Białej",22,23,Stawowej,stawowy:A,adj:sg:dat:f:pos,[],[],stawowy
165831,2020,OLSZTYN,146. Biesiady Pieczewskie 2021 - otwarta impreza plenerowa,2,3,Biesiady,Biesiada:Sf,subst:sg:gen:f,['nazwisko'],[],Biesiada
284991,2018/19,ŁÓDŹ,ODNOWA TEOFILOWA: Modernizacja ul. Łanowej - zielone miejsca postojowe.,6,7,Łanowej,Łanowa,subst:sg:dat.loc:f,['nazwisko'],[],Łanowa
231550,2020,WROCŁAW,Park rekreacyjny na Psim Polu etap III,4,5,Polu,Pol:Sm1,subst:sg:voc:m1,['nazwisko'],[],Pol
105911,2019,KATOWICE,L2/11/VI - Kontynuacja zadan remontowych polegajacych na wymianie i budowie nowych chodników wraz Z latarniami,4,5,VI,VI,romandig,[],[],VI


In [29]:
df_roc['tokens'] = df_roc['lemma_clean'].apply(simple_preprocess)
df_roc.sample(20)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_,lemma_clean,tokens
195675,2020,POZNAŃ,"Festiwal muzyczno-kulturalny ""Piątkowo Open Air""",2,3,-,-,interp,[],[],-,[]
301083,2018/19,ŁÓDŹ,"Mały sport - wielka radość w SP nr 199 im. J. Tuwima, ul. Elsnera 8 - remont i modernizacja sali gimnastycznej.",15,16,ul,ul,subst:sg:nom.acc:m3,['nazwa_pospolita'],[],ul,[ul]
191716,2019,POZNAŃ,Remont placu zabaw- wewnętrzny dziedziniec szkolny,3,4,-,-,interp,[],[],-,[]
152845,2018,KIELCE,"Budowa parkingu dla 40 samochodów osobowych na Osiedlu ""Ślichowice II"" wraz z niezbędną infrastrukturą",1,2,parkingu,parking,subst:sg:gen:m3,['nazwa_pospolita'],[],parking,[parking]
376661,2019,KRAKÓW,"Koncerty - Jesień z ""Muzyką i Sacrum""",5,6,Muzyką,Muzyka:Sf,subst:sg:inst:f,['nazwisko'],[],Muzyka,[muzyka]
343552,2020,ŁÓDŹ,Kwitnące żywopłoty dla Łodzi – sadzimy piękne krzewy na Broniewskiego.,0,1,Kwitnące,kwitnąć,pact:sg:nom.acc.voc:n:imperf:aff,[],[],kwitnąć,[kwitnąć]
193530,2019,POZNAŃ,"Ratujemy S/Y ""Kołobrzeg""",1,2,S/Y,sail_yacht,brev:npun,[],['mors.'],sail_yacht,[sail_yacht]
53925,2018,BIELSKO-BIAŁA,Wykonanie placu zabaw w rejonie ulicy Asnyka w Bielsku-Białej,8,9,Bielsku,Bielsko,subst:sg:dat:n:ncol,['nazwa_geograficzna'],[],Bielsko,[bielsko]
302875,2018/19,ŁÓDŹ,"Przystanek Arturówek, edukacja historyczna dla wszystkich.",6,7,wszystkich,wszystek,adj:pl:loc:m1.m2.m3.f.n:pos,[],[],wszystek,[wszystek]
163723,2020,ELBLĄG,"Naprawa lub częściowa wymiana chodnika, wykonanie zatoczek parkingowych i częściowa naprawa drogi wzdłuż ulicy Ogrodowej",8,9,parkingowych,parkingowy:A,adj:pl:loc:m1.m2.m3.f.n:pos,[],[],parkingowy,[parkingowy]


In [30]:
df_roc3 = df_roc[df_roc["tokens"].str.len() != 0]
df_roc3.sample(60)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_,lemma_clean,tokens
412646,2020,KRAKÓW,taczymy Parki Krakowa (kontynuacja) - stacje naprawy rowerów przy trasach dla rowerzystów,0,1,taczymy,taczymy,ign,[],[],taczymy,[taczymy]
382682,2019,NOWY SĄCZ,Dobry parking na Barskim,1,2,parking,parking,subst:sg:nom.acc:m3,['nazwa_pospolita'],[],parking,[parking]
353704,2020,ŁÓDŹ,"Modernizacja szatni, w tym pomieszczenia na szatnię dla uczniów niepełnosprawnych, w XXXIII",5,6,pomieszczenia,pomieścić,ger:pl:nom.acc:n:perf:aff,[],[],pomieścić,[pomieścić]
184510,2019,KONIN,NASADZENIA KRZEWÓW KWITNACYCH W KONINIE,4,5,KONINIE,Konin,subst:sg:loc:m3,['nazwa_geograficzna'],[],Konin,[konin]
74526,2020,CZĘSTOCHOWA,Zielony pochlaniacz smogu dla kazdej dzielnicy.,0,1,Zielony,Zielony:Sf,subst:sg.pl:nom.gen.dat.acc.inst.loc.voc:f,['nazwisko'],[],Zielony,[zielony]
356830,2020,ŁÓDŹ,USTAWIENIE BRAKUJĄCYCH LATARNI W PARKU POMIĘDZY UL. LUTOMIERSKĄ A UL. DREWNOWSKĄ.,2,3,LATARNI,latarnia,subst:sg:dat.loc:f,['nazwa_pospolita'],[],latarnia,[latarnia]
234390,2018,BYDGOSZCZ,Budowa zatok postojowych w obrębie drogi miejskiej ul. Monte Cassino,2,3,postojowych,postojowe,subst:pl:loc:n:ncol,['nazwa_pospolita'],[],postojowe,[postojowe]
25486,2020,GDAŃSK,Kontynuacja ciągu spacerowo-rowerowy nad Martwą Wisłą do połączenia z ul. Nadwiślańską,6,7,Martwą,martwy:A,adj:sg:inst:f:pos,[],[],martwy,[martwy]
22624,2020,GDAŃSK,Koty - Pomóżmy im.,2,3,Pomóżmy,pomóc,impt:pl:pri:perf,[],[],pomóc,[pomóc]
5641,2020,GDYNIA,"Łąki kwietne w Redłowie - estetyzacja podwórek w obrębie ulic Powstania Wielkopolskiego, Powstania Śląskiego",5,6,estetyzacja,estetyzacja,subst:sg:nom:f,['nazwa_pospolita'],[],estetyzacja,[estetyzacja]


In [31]:
#cbow
processed_docs = df_roc3['tokens']
vocab = gensim.corpora.Dictionary(processed_docs)
bow_corpus = [vocab.doc2bow(doc) for doc in processed_docs]

lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=5, id2word=vocab, passes=2)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
    print("")

Topic: 0 
Words: 0.075*"ulica" + 0.052*"dla" + 0.037*"miejski" + 0.024*"dzielnica" + 0.022*"numer" + 0.022*"sportowy" + 0.016*"on" + 0.012*"mały" + 0.012*"to" + 0.010*"sala"

Topic: 1 
Words: 0.049*"do" + 0.026*"dziecko" + 0.022*"zabawić" + 0.018*"stary" + 0.018*"miasto" + 0.018*"sp" + 0.017*"remont" + 0.011*"nie" + 0.011*"dobry" + 0.011*"parkingowy"

Topic: 2 
Words: 0.068*"nowy" + 0.053*"ul" + 0.044*"przy" + 0.028*"zabawa" + 0.020*"podstawowy" + 0.016*"remonta" + 0.016*"oraz" + 0.013*"zieleń" + 0.012*"budynek" + 0.011*"młodzież"

Topic: 3 
Words: 0.086*"zielony" + 0.054*"park" + 0.029*"nowa" + 0.027*"centrum" + 0.026*"łódź" + 0.023*"pieszy" + 0.020*"zielone" + 0.020*"wolny" + 0.020*"nawierzchnia" + 0.019*"budowa"

Topic: 4 
Words: 0.214*"na" + 0.046*"plac" + 0.040*"osiedle" + 0.035*"szkoła" + 0.034*"mini" + 0.029*"budowo" + 0.026*"bezpieczny" + 0.022*"chodnik" + 0.021*"nr" + 0.010*"rekreacyjny"



In [51]:
#tfidf
processed_docs = df_roc3['tokens']
vocab = gensim.corpora.Dictionary(processed_docs)
corpus = [vocab.doc2bow(doc) for doc in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=5, id2word=vocab, passes=2, workers=10)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.045*"park" + 0.027*"zabawa" + 0.023*"bezpieczny" + 0.021*"nr" + 0.018*"nowa" + 0.012*"wraz" + 0.010*"oda" + 0.010*"filia" + 0.009*"ścieżka" + 0.009*"poprawa"
Topic: 1 
Words: 0.206*"na" + 0.035*"miejski" + 0.023*"chodnik" + 0.022*"numer" + 0.019*"teren" + 0.018*"podstawowy" + 0.014*"miejsce" + 0.013*"droga" + 0.013*"parking" + 0.012*"miasto"
Topic: 2 
Words: 0.066*"ul" + 0.030*"szkoła" + 0.024*"oraz" + 0.020*"centrum" + 0.015*"sp" + 0.015*"zielone" + 0.015*"drogi" + 0.012*"mały" + 0.011*"to" + 0.011*"rekreacyjny"
Topic: 3 
Words: 0.061*"zielony" + 0.049*"nowy" + 0.046*"plac" + 0.035*"łódź" + 0.032*"mini" + 0.022*"dziecko" + 0.022*"budowa" + 0.019*"rowerowy" + 0.017*"stary" + 0.014*"boisko"
Topic: 4 
Words: 0.092*"ulica" + 0.056*"do" + 0.051*"dla" + 0.049*"przy" + 0.036*"budowo" + 0.031*"pieszy" + 0.029*"osiedle" + 0.021*"sportowy" + 0.021*"zabawić" + 0.020*"nawierzchnia"


In [33]:
tags = ['prep', 'conj', 'interj', 'part', 'adv']
df_roc_clean = df_roc3[~df_roc3['tag'].str.contains('|'.join(tags))]
df_roc_clean.sample(60)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_,lemma_clean,tokens
77668,2018,DĄBROWA GÓRNICZA,Kocie towarzystwo na Starym Gotonogu,0,1,Kocie,kot:Sm1,subst:sg:voc:m1,['nazwa_pospolita'],"['pot.,środ.']",kot,[kot]
209414,2019,ŚWINOUJŚCIE,obcowania Z róznymi dziedzinami sztuki. Odkrywanie przez ucznia zagadnien muzycznych i plastycznych glownie poprzez dzialania praktyczne. Stworzenie indywidualnych miejsc pracy wraz Z zapleczem instrumentów. Stanowisko nauczyciela wyposazone W sprzet multimedialny. Funkcja auli do kameralnych apeli i lekcji pokazowych oraz przedstawien.,16,17,praktyczne,praktyczny,adj:sg:nom.voc:n:pos,[],[],praktyczny,[praktyczny]
138207,2019,SOSNOWIEC,Uzupelnienie obiektów sportowych przy Szkole Podstawowej nr 20 im. Mikotaja Kopernika W Sosnowcu,2,3,sportowych,sportowy,adj:pl:loc:m1.m2.m3.f.n:pos,[],[],sportowy,[sportowy]
325939,2019,ŁÓDŹ,Ograniczanie bezdomności zwierząt na terenie osiedla Złotno.,5,6,osiedla,osiedlać,fin:sg:ter:imperf,[],[],osiedlać,[osiedlać]
194864,2019,POZNAŃ,Odbudowa południowych klinów zieleni- Chartowo kontynuacja,6,7,kontynuacja,kontynuacja,subst:sg:nom:f,['nazwa_pospolita'],[],kontynuacja,[kontynuacja]
270619,2019,LUBLIN,Zwiększenie funkcjonalności boiska trawiastego Lubelskiego Centrum Kształcenia Zawodowego i Ustawicznego,6,7,Kształcenia,kształcenie,subst:pl:nom.acc.voc:n:ncol,['nazwa_pospolita'],[],kształcenie,[kształcenie]
151171,2020,ŻORY,Zazielenić Żory,1,2,Żory,Żory,subst:pl:nom.acc.voc:n:pt,['nazwa_geograficzna'],[],Żory,[żory]
290940,2018/19,ŁÓDŹ,Zamiast błotka i PRL-owskich betonowych płyt bezpieczne i estetyczne przejścia i chodnik.,1,2,błotka,błotko,subst:pl:nom.acc.voc:n:ncol,['nazwa_pospolita'],[],błotko,[błotko]
204161,2018,SZCZECIN,Miejskie poidełka na Jasnych Błoniach,4,5,Błoniach,błoń,subst:pl:loc:f,['nazwa_pospolita'],['przest.'],błoń,[błoń]
96731,2020,KATOWICE,"Biblioteka – miejsce dla całej rodziny: nowości wydawnicze dla filii nr 17 (książki, audiobooki, gry planszowe). Doposażenie filii i dofinansowanie warsztatów „kreatywna Biblioteka”",8,9,wydawnicze,wydawniczy,adj:sg:acc:n:pos,[],[],wydawniczy,[wydawniczy]


In [34]:
#cbow
processed_docs = df_roc_clean['tokens']
vocab = gensim.corpora.Dictionary(processed_docs)
bow_corpus = [vocab.doc2bow(doc) for doc in processed_docs]


lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=5, id2word=vocab, passes=2)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
    print("")

Topic: 0 
Words: 0.043*"miejski" + 0.043*"szkoła" + 0.036*"budowo" + 0.035*"łódź" + 0.026*"nr" + 0.023*"wolny" + 0.021*"sp" + 0.016*"boisko" + 0.015*"miasto" + 0.013*"młodzież"

Topic: 1 
Words: 0.063*"park" + 0.031*"zabawa" + 0.027*"pieszy" + 0.026*"do" + 0.023*"zielone" + 0.020*"stary" + 0.018*"on" + 0.014*"parking" + 0.012*"miejsce" + 0.011*"orlik"

Topic: 2 
Words: 0.049*"plac" + 0.044*"osiedle" + 0.036*"mini" + 0.032*"nowa" + 0.028*"bezpieczny" + 0.026*"dziecko" + 0.025*"dzielnica" + 0.023*"numer" + 0.017*"remont" + 0.011*"zielona"

Topic: 3 
Words: 0.081*"ulica" + 0.031*"centrum" + 0.024*"sportowy" + 0.023*"zabawić" + 0.022*"podstawowy" + 0.022*"budowa" + 0.020*"teren" + 0.018*"remonta" + 0.014*"rowerowy" + 0.013*"mały"

Topic: 4 
Words: 0.094*"zielony" + 0.069*"nowy" + 0.054*"ul" + 0.024*"chodnik" + 0.021*"nawierzchnia" + 0.015*"huta" + 0.013*"zieleń" + 0.012*"budynek" + 0.011*"modernizacja" + 0.011*"sala"



In [48]:
#tfidf
processed_docs = df_roc_clean['tokens']
vocab = gensim.corpora.Dictionary(processed_docs)
corpus = [vocab.doc2bow(doc) for doc in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=5, id2word=vocab, passes=2, workers=10)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.055*"nowy" + 0.038*"budowo" + 0.033*"szkoła" + 0.026*"chodnik" + 0.023*"sportowy" + 0.022*"nawierzchnia" + 0.020*"remont" + 0.020*"remonta" + 0.016*"drogi" + 0.016*"miejsce"
Topic: 1 
Words: 0.098*"ulica" + 0.037*"mini" + 0.025*"bezpieczny" + 0.024*"budowa" + 0.023*"numer" + 0.021*"zabawić" + 0.020*"podstawowy" + 0.017*"dzielnica" + 0.016*"zieleń" + 0.015*"on"
Topic: 2 
Words: 0.053*"plac" + 0.022*"centrum" + 0.021*"nowa" + 0.018*"parkingowy" + 0.016*"boisko" + 0.013*"wraz" + 0.012*"biblioteka" + 0.011*"modernizacja" + 0.011*"filia" + 0.010*"to"
Topic: 3 
Words: 0.075*"zielony" + 0.070*"ul" + 0.050*"park" + 0.036*"miejski" + 0.036*"łódź" + 0.025*"dziecko" + 0.022*"nr" + 0.017*"zielone" + 0.013*"oraz" + 0.012*"budynek"
Topic: 4 
Words: 0.032*"pieszy" + 0.030*"osiedle" + 0.030*"zabawa" + 0.030*"do" + 0.022*"rowerowy" + 0.020*"teren" + 0.019*"stary" + 0.017*"sp" + 0.015*"droga" + 0.010*"przedszkole"


In [49]:
# Wizualizacja tematów
vis_data = vis.prepare(lda_model, corpus_tfidf, vocab)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis_data)

In [50]:
#tfidf
processed_docs = df_roc_clean['tokens']
vocab = gensim.corpora.Dictionary(processed_docs)
corpus = [vocab.doc2bow(doc) for doc in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=20, id2word=vocab, passes=2, workers=10)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.095*"dziecko" + 0.095*"bezpieczny" + 0.085*"nowa" + 0.085*"centrum" + 0.078*"dzielnica" + 0.027*"aktywny" + 0.024*"drzewo" + 0.021*"przejść" + 0.018*"nasz" + 0.017*"zielenić"
Topic: 1 
Words: 0.050*"budynek" + 0.042*"poprawa" + 0.039*"to" + 0.029*"mieszkaniec" + 0.028*"publiczny" + 0.028*"polski" + 0.026*"ruch" + 0.025*"część" + 0.019*"staw" + 0.019*"stacja"
Topic: 2 
Words: 0.085*"remonta" + 0.056*"parking" + 0.026*"odcinek" + 0.022*"bezpieczeństwo" + 0.020*"przychodni" + 0.016*"integracyjny" + 0.015*"ogrodzić" + 0.015*"wszyscy" + 0.014*"rucho" + 0.013*"ogrodzenie"
Topic: 3 
Words: 0.086*"rowerowy" + 0.030*"pies" + 0.026*"zająć" + 0.025*"rewitalizacja" + 0.022*"stara" + 0.019*"zagospodarować" + 0.017*"zespół" + 0.016*"smok" + 0.015*"ścieżek" + 0.013*"nasadzić"
Topic: 4 
Words: 0.206*"nowy" + 0.133*"budowo" + 0.120*"szkoła" + 0.027*"książka" + 0.024*"szkolny" + 0.023*"profesjonalny" + 0.020*"psi" + 0.019*"letni" + 0.015*"biegowy" + 0.012*"asfaltowy"
Topic: 5 
Words: 

In [83]:
stopwords = ['do', 'wraz', 'oraz', 'ul', 'ulica', 'nr', 'numer', 'to', 'on', 'przy', 'dla', 'na', 'po','jak', 'ten', 'nie', 'od', 'oda', 'czyli', 'ii']
df_roc_clean4 = df_roc3[~df_roc3['lemma'].str.contains('|'.join(stopwords))]
df_roc_clean4.sample(10)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_,lemma_clean,tokens
69366,2019,CZĘSTOCHOWA,"Budowa boiska typu ""Orlik""",4,5,Orlik,Orlik:Sf,subst:sg.pl:nom.gen.dat.acc.inst.loc.voc:f,['nazwisko'],[],Orlik,[orlik]
178666,2018,KALISZ,Zakup stołu do tenisa stołowego dla osiedla Korczak.,7,8,Korczak,Korczak:Sm1,subst:sg:nom:m1,['nazwisko'],[],Korczak,[korczak]
336330,2019,ŁÓDŹ,Rodzinna Strefa Gier Podwórkowych na Starym Widzewie.,2,3,Gier,gra,subst:pl:gen:f,['nazwa_pospolita'],[],gra,[gra]
265195,2020,CHEŁM,Modernizacja placu zabaw przy Przedszkolu Miejskim Nr 11 w Chełmie,2,3,zabaw,zabawa,subst:pl:gen:f,['nazwa_pospolita'],[],zabawa,[zabawa]
108019,2019,KATOWICE,"L9/10/VI - Przedszkolaka krok ku bezpiecznej przysztosci - doposazenie miejskich przedszkoli nr 21, 25, 34, 57 na Osiedlu Tysiaclecia",24,25,Osiedlu,osiedle,subst:sg:dat:n:ncol,['nazwa_pospolita'],[],osiedle,[osiedle]
302151,2018/19,ŁÓDŹ,OSIEDLOWE CENTRUM KULTURY I INTEGRACJI PRZY OSP ŁÓDŻ – NOWOSOLNA.,1,2,CENTRUM,centrum,subst:sg:dat:n:ncol,['nazwa_pospolita'],[],centrum,[centrum]
154523,2019,KIELCE,",,Magiczny Ogród- Pozwólmy się dzieciom bawić""- budowa i modernizacja placu zabaw dla przedszkolaków i mieszkańców Malikowa i okolic. Powiększenie placu zabaw, poprawienie bezpieczeństwa poprzez wymianę ogrodzenia i bramy.",7,8,dzieciom,dziecko,subst:pl:dat:n:col,['nazwa_pospolita'],[],dziecko,[dziecko]
267876,2018,LUBLIN,Lubelska akademia koszykówki,0,1,Lubelska,Lubelska,subst:sg:nom:f,['nazwisko'],[],Lubelska,[lubelska]
265693,2018,LUBLIN,Rozbudowa bazy sportowej w XXX Liceum Ogólnokształcącym z oddziałami gimnazjalnymi im. księdza Jana Twardowskiego w Lublinie,1,2,bazy,baza,subst:sg:gen:f,['nazwa_pospolita'],[],baza,[baza]
148508,2020,ZABRZE,P0041 Ścieżka pomiędzy basenem a ogrodami z osiedla Dzierżona do kościoła pw. św. Krzyża,1,2,Ścieżka,ścieżek,subst:sg:gen:m3,['nazwa_pospolita'],[],ścieżek,[ścieżek]


In [85]:
#tfidf
processed_docs = df_roc_clean4['tokens']
vocab = gensim.corpora.Dictionary(processed_docs)
corpus = [vocab.doc2bow(doc) for doc in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=6, id2word=vocab, passes=2, workers=10)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.061*"mini" + 0.020*"mały" + 0.018*"budynek" + 0.015*"filia" + 0.015*"przedszkole" + 0.012*"nowość" + 0.011*"aktywny" + 0.011*"ogród" + 0.010*"książka" + 0.010*"nowe"
Topic: 1 
Words: 0.055*"łódź" + 0.039*"nowa" + 0.037*"centrum" + 0.029*"parkingowy" + 0.025*"droga" + 0.018*"zakup" + 0.016*"wzdłuż" + 0.013*"pies" + 0.013*"przestrzeń" + 0.012*"rekreacja"
Topic: 2 
Words: 0.084*"plac" + 0.044*"bezpieczny" + 0.036*"dzielnica" + 0.028*"sp" + 0.027*"boisko" + 0.014*"miejska" + 0.014*"oświetlić" + 0.013*"osiedlowy" + 0.012*"wielofunkcyjny" + 0.011*"edukacyjny"
Topic: 3 
Words: 0.101*"nowy" + 0.064*"miejski" + 0.050*"osiedle" + 0.044*"dziecko" + 0.036*"stary" + 0.029*"zieleń" + 0.027*"miejsce" + 0.026*"drogi" + 0.023*"parking" + 0.018*"sala"
Topic: 4 
Words: 0.093*"park" + 0.053*"zabawa" + 0.036*"teren" + 0.036*"rowerowy" + 0.014*"siłownia" + 0.014*"wolny" + 0.011*"zająć" + 0.011*"szkolny" + 0.010*"rok" + 0.010*"im"
Topic: 5 
Words: 0.056*"pieszy" + 0.053*"szkoła" + 0.036*"z

In [89]:
df_roc_clean4[df_roc_clean4['tag']=='adv'].sample(40)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_,lemma_clean,tokens
2588,2020,GDYNIA,Ławki z bali (pni) na trasach spacerowych w lasach i trasy rowerowe dla rowerów BMX „ENDURO” obok parku rodzinnego przy ul. Spokojnej,20,21,obok,obok:D,adv,[],[],obok,[obok]
383190,2020,NOWY SĄCZ,Parking dla samochodów obok Szkoły Podstawowej nr 18.,3,4,obok,obok:D,adv,[],[],obok,[obok]
265971,2018,LUBLIN,Budowa ścieżki rowerowej wzdłuż al. Solidarności od stacji obsługi PZM w kierunku podstacji gazowej i restauracji Locomotiva,3,4,wzdłuż,wzdłuż:D,adv,[],[],wzdłuż,[wzdłuż]
401760,2018,TARNOBRZEG,"Remont chodnika wzdłuż ul. Św. Barbary, prawa strona do ul. Sikorskiego.",2,3,wzdłuż,wzdłuż:D,adv,[],[],wzdłuż,[wzdłuż]
37527,2018,GDAŃSK,Gdańsk tu oddycham. System czujników jakości powietrza w każdej dzielnicy,1,2,tu,tu,adv,[],[],tu,[tu]
285706,2018/19,ŁÓDŹ,"„Gdy twój rower daszek znajdzie, na siłowni będzie fajniej""- montaż zadaszonych stojaków na rowery oraz siłowni przy SP 120.",1,2,Gdy,gdy:D,adv,[],[],gdy,[gdy]
238224,2020,BYDGOSZCZ,Posadzenie drzewek przy ulicy Zamenhofa (ul. przy tyle sklepu E.Leclerc),9,10,tyle,tyle:D,adv,[],[],tyle,[tyle]
234262,2018,BYDGOSZCZ,"Wyznaczenie stref, w których możliwe jest picie alkoholou wzdłuż Kanału Bydgoskiego",9,10,wzdłuż,wzdłuż:D,adv,[],[],wzdłuż,[wzdłuż]
98304,2020,KATOWICE,"miejscach, gdzie beda dalej petnic swoja funkcjq edukacyjna. Zadanie ma na celu rewitalizacje zywoptotu przy ul. 1 Maja na wysokosci ul. Murckowskiej W dzielnicy Zawodzie. W ramach zadania zielen powinna zostac uzupelniona W miejscach ubytków, Z zachowaniem zasad bezpieczenstwa dla ruchu pieszego i widocznosci pojazdów poruszajacych siq po ul. 1 Maja. Planowane jest, aby nasadzenia skladafy si...",2,3,gdzie,gdzie:D,adv,[],[],gdzie,[gdzie]
154944,2019,KIELCE,Budowa bieżni lekkoatletycznej dwutorowej zakończonej zeskokiem do skoku w dal wraz z zagospodarowaniem terenu wokół niej.,14,15,wokół,wokół:D,adv,[],[],wokół,[wokół]


## 2018

In [39]:
df_2018 = pd.read_csv('/content/drive/My Drive/Konkursy Kaggle/Budżet obywatelski/Morfeusz_2018.csv')
df_2018.head()

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_
0,2018,GDYNIA,"Zagospodarowanie terenu po bytym baraku przy ulicy Dedala - stworzenie sciezek, ustawienie fawek, nasadzenia zieleni i aranzacja matej architektury",0,1,Zagospodarowanie,zagospodarować,ger:sg:nom.acc:n:perf:aff,[],[]
1,2018,GDYNIA,"Zagospodarowanie terenu po bytym baraku przy ulicy Dedala - stworzenie sciezek, ustawienie fawek, nasadzenia zieleni i aranzacja matej architektury",1,2,terenu,teren,subst:sg:gen:m3,['nazwa_pospolita'],[]
2,2018,GDYNIA,"Zagospodarowanie terenu po bytym baraku przy ulicy Dedala - stworzenie sciezek, ustawienie fawek, nasadzenia zieleni i aranzacja matej architektury",2,3,po,po,prep:acc,[],[]
3,2018,GDYNIA,"Zagospodarowanie terenu po bytym baraku przy ulicy Dedala - stworzenie sciezek, ustawienie fawek, nasadzenia zieleni i aranzacja matej architektury",2,3,po,po,prep:dat,[],[]
4,2018,GDYNIA,"Zagospodarowanie terenu po bytym baraku przy ulicy Dedala - stworzenie sciezek, ustawienie fawek, nasadzenia zieleni i aranzacja matej architektury",2,3,po,po,prep:loc,[],[]


In [41]:
df_2018['lemma_clean'] = df_2018['lemma'].str.split(':', expand=True)[0]
df_2018['tokens'] = df_2018['lemma_clean'].apply(simple_preprocess)
df_2018_ = df_2018[df_2018["tokens"].str.len() != 0]

In [43]:
#tfidf
processed_docs = df_2018_['tokens']
vocab = gensim.corpora.Dictionary(processed_docs)
corpus = [vocab.doc2bow(doc) for doc in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=5, id2word=vocab, passes=2, workers=10)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.060*"do" + 0.054*"dla" + 0.036*"pieszy" + 0.031*"szkoła" + 0.030*"miejski" + 0.030*"dziecko" + 0.028*"zabawa" + 0.022*"nr" + 0.020*"parkingowy" + 0.020*"centrum"
Topic: 1 
Words: 0.066*"zielony" + 0.032*"oraz" + 0.029*"rowerowy" + 0.028*"numer" + 0.026*"nawierzchnia" + 0.026*"zabawić" + 0.020*"teren" + 0.020*"zieleń" + 0.017*"młodzież" + 0.013*"rekreacyjny"
Topic: 2 
Words: 0.053*"przy" + 0.053*"plac" + 0.043*"mini" + 0.022*"chodnik" + 0.018*"bezpieczny" + 0.014*"remont" + 0.011*"wzdłuż" + 0.010*"publiczny" + 0.009*"oświetlić" + 0.009*"sp"
Topic: 3 
Words: 0.195*"na" + 0.060*"ul" + 0.047*"nowy" + 0.040*"park" + 0.020*"sportowy" + 0.020*"osiedle" + 0.016*"nowa" + 0.015*"on" + 0.014*"drogi" + 0.014*"dorosły"
Topic: 4 
Words: 0.078*"ulica" + 0.029*"budowo" + 0.020*"budowa" + 0.019*"podstawowy" + 0.016*"stary" + 0.014*"mały" + 0.014*"boisko" + 0.012*"droga" + 0.012*"dzielnica" + 0.011*"po"


In [45]:
tags = ['prep', 'conj', 'interj', 'part']
df_2018_clean = df_2018_[~df_2018_['tag'].str.contains('|'.join(tags))]
df_2018_clean.sample(10)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_,lemma_clean,tokens
72407,2018,ZAMOŚĆ,"Wykonanie placu zabaw między budynkami przy ul, Płk. Lisa Kuli 4 i ul. Generała Maczka 15 wrraz z zagospodarowaniem otoczenia",22,23,otoczenia,otoczenie,subst:sg:gen:n:ncol,['nazwa_pospolita'],[],otoczenie,[otoczenie]
62747,2018,GRUDZIĄDZ,Wykonanie remontu nawierzchni ulicy Różanej w Grudziądzu,4,5,Różanej,różany,adj:sg:gen:f:pos,[],[],różany,[różany]
300247,2018,WROCŁAW,Park Leonarda da Vinci na Gądowie Małym,6,7,Małym,mały:S,subst:sg:inst:m1,['nazwa_pospolita'],[],mały,[mały]
116317,2018,WARSZAWA,Psi Tor Przeszkód,2,3,Przeszkód,Przeszkoda,subst:pl:gen:f,['nazwa_geograficzna'],[],Przeszkoda,[przeszkoda]
184319,2018,GRUDZIĄDZ,Wybudowanie chodnika wzdłuż ulicy Dębowej na odcinku od ul. Wierzbowej w kierunku do ul. Akacjowej,10,11,Wierzbowej,wierzbowy,adj:sg:dat:f:pos,[],[],wierzbowy,[wierzbowy]
37920,2018,OLSZTYN,43. Czytamy w Teatrze Nowym w Olsztynie,4,5,Teatrze,teatr,subst:sg:loc:m3,['nazwa_pospolita'],[],teatr,[teatr]
226023,2018,WARSZAWA,Budowa chodnika pomiędzy osiedlem Wilanowska/Domaniewska a przejściem dla pieszych przy Al. Niepodległości,0,1,Budowa,budowa,subst:sg:nom:f,['nazwa_pospolita'],[],budowa,[budowa]
255737,2018,CZĘSTOCHOWA,Budowa mini kompleksu rekreacyjno - rehabilitacyjnego W Parku Piastów,1,2,mini,mini:A,adj:sg:gen:f:pos,[],[],mini,[mini]
137834,2018,JASTRZĘBIE ZDRÓJ,Zielony wypoczynek - dostosowanie do potrzeb mieszkańców terenu rekreacyjnego w rejonie SP-13 i boiska MOSIR przy ul.Katowickiej,11,12,SP,SP,subst:sg.pl:nom.gen.dat.acc.inst.loc.voc:f,['nazwa_organizacji'],[],SP,[sp]
343480,2018,WARSZAWA,"Urządzenie zieleni i posadzenie drzew po północnej stronie ulicy Kijowskiej, na wysokości Dworca Wschodniego .",1,2,zieleni,zieleń,subst:pl:gen:f,['nazwa_pospolita'],[],zieleń,[zieleń]


In [46]:
#tfidf
processed_docs = df_2018_clean['tokens']
vocab = gensim.corpora.Dictionary(processed_docs)
corpus = [vocab.doc2bow(doc) for doc in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=5, id2word=vocab, passes=2, workers=10)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.044*"mini" + 0.030*"szkoła" + 0.028*"zabawa" + 0.021*"parkingowy" + 0.020*"osiedle" + 0.015*"on" + 0.015*"młodzież" + 0.011*"parking" + 0.011*"siłownia" + 0.010*"bezpłatny"
Topic: 1 
Words: 0.074*"ul" + 0.065*"plac" + 0.026*"nr" + 0.026*"nawierzchnia" + 0.024*"sportowy" + 0.023*"centrum" + 0.021*"miejsce" + 0.020*"nowa" + 0.020*"zieleń" + 0.020*"teren"
Topic: 2 
Words: 0.048*"park" + 0.035*"budowo" + 0.027*"rowerowy" + 0.026*"chodnik" + 0.016*"remont" + 0.016*"remonta" + 0.015*"droga" + 0.013*"wraz" + 0.012*"zielone" + 0.012*"zająć"
Topic: 3 
Words: 0.053*"zielony" + 0.037*"pieszy" + 0.029*"miejski" + 0.023*"numer" + 0.022*"budowa" + 0.017*"dorosły" + 0.013*"warszawa" + 0.010*"modernizacja" + 0.010*"rekreacyjny" + 0.010*"oświetlić"
Topic: 4 
Words: 0.107*"ulica" + 0.057*"nowy" + 0.035*"do" + 0.034*"dziecko" + 0.025*"zabawić" + 0.025*"podstawowy" + 0.022*"stary" + 0.022*"bezpieczny" + 0.018*"mały" + 0.015*"oraz"


In [47]:
# Wizualizacja tematów
vis_data = vis.prepare(lda_model, corpus_tfidf, vocab)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis_data)

## 2019

In [54]:
df_2019 = pd.read_csv('/content/drive/My Drive/Konkursy Kaggle/Budżet obywatelski/Morfeusz_2019.csv')
df_2019.head()

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_
0,2019,GDYNIA,"Parki kieszonkowe W 9 dzielnicach: Dqbrowa, Grabówek, Wzgórze Swietego Maksymiliana, Chylonia, Pogórze, Dziatki Lesne, Leszczynki, Pustki Cisowskie, Wielki Kack",0,1,Parki,park,subst:pl:nom.acc.voc:m3,['nazwa_pospolita'],[]
1,2019,GDYNIA,"Parki kieszonkowe W 9 dzielnicach: Dqbrowa, Grabówek, Wzgórze Swietego Maksymiliana, Chylonia, Pogórze, Dziatki Lesne, Leszczynki, Pustki Cisowskie, Wielki Kack",0,1,Parki,parka,subst:sg:gen:f,['nazwa_pospolita'],[]
2,2019,GDYNIA,"Parki kieszonkowe W 9 dzielnicach: Dqbrowa, Grabówek, Wzgórze Swietego Maksymiliana, Chylonia, Pogórze, Dziatki Lesne, Leszczynki, Pustki Cisowskie, Wielki Kack",0,1,Parki,parka,subst:pl:nom.acc.voc:f,['nazwa_pospolita'],[]
3,2019,GDYNIA,"Parki kieszonkowe W 9 dzielnicach: Dqbrowa, Grabówek, Wzgórze Swietego Maksymiliana, Chylonia, Pogórze, Dziatki Lesne, Leszczynki, Pustki Cisowskie, Wielki Kack",0,1,Parki,Parka:Sf,subst:sg:gen:f,"['imię', 'nazwisko']",[]
4,2019,GDYNIA,"Parki kieszonkowe W 9 dzielnicach: Dqbrowa, Grabówek, Wzgórze Swietego Maksymiliana, Chylonia, Pogórze, Dziatki Lesne, Leszczynki, Pustki Cisowskie, Wielki Kack",0,1,Parki,Parka:Sf,subst:pl:nom.acc.voc:f,"['imię', 'nazwisko']",[]


In [55]:
df_2019['lemma_clean'] = df_2019['lemma'].str.split(':', expand=True)[0]
df_2019['tokens'] = df_2019['lemma_clean'].apply(simple_preprocess)
df_2019_ = df_2019[df_2019["tokens"].str.len() != 0]

In [56]:
#tfidf
processed_docs = df_2019_['tokens']
vocab = gensim.corpora.Dictionary(processed_docs)
corpus = [vocab.doc2bow(doc) for doc in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=5, id2word=vocab, passes=2, workers=10)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.054*"ul" + 0.047*"plac" + 0.044*"park" + 0.040*"przy" + 0.030*"dziecko" + 0.017*"nawierzchnia" + 0.015*"zieleń" + 0.014*"remont" + 0.010*"wraz" + 0.010*"budynek"
Topic: 1 
Words: 0.075*"ulica" + 0.051*"zielony" + 0.050*"nowy" + 0.019*"zabawić" + 0.018*"numer" + 0.017*"nowa" + 0.015*"parkingowy" + 0.015*"boisko" + 0.014*"młodzież" + 0.012*"sp"
Topic: 2 
Words: 0.200*"na" + 0.046*"do" + 0.027*"zabawa" + 0.023*"budowo" + 0.017*"stary" + 0.014*"teren" + 0.013*"on" + 0.012*"zielone" + 0.009*"zakup" + 0.008*"stare"
Topic: 3 
Words: 0.030*"mini" + 0.027*"pieszy" + 0.024*"miejski" + 0.021*"bezpieczny" + 0.021*"chodnik" + 0.020*"sportowy" + 0.020*"rowerowy" + 0.018*"nr" + 0.017*"dzielnica" + 0.015*"remonta"
Topic: 4 
Words: 0.052*"dla" + 0.025*"oraz" + 0.025*"szkoła" + 0.019*"osiedle" + 0.018*"budowa" + 0.017*"podstawowy" + 0.013*"miejsce" + 0.012*"drogi" + 0.010*"zajęcie" + 0.008*"po"


In [57]:
tags = ['prep', 'conj', 'interj', 'part']
df_2019_clean = df_2019_[~df_2019_['tag'].str.contains('|'.join(tags))]
df_2019_clean.sample(10)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_,lemma_clean,tokens
83174,2019,BYDGOSZCZ,Rewitalizcja terenu w koło placu zabaw.,3,4,koło,koło:S,subst:sg:nom.acc.voc:n:ncol,['nazwa_pospolita'],[],koło,[koło]
112147,2019,ŁÓDŹ,Bezpieczne przejście dla pieszych na wysokości ul. Traktorowej 13 (przy Mikomedzie).,3,4,pieszych,pieszy:S,subst:pl:gen.acc:m1,['nazwa_pospolita'],[],pieszy,[pieszy]
145020,2019,BIAŁYSTOK,Ochrona drzew przy ul. Marii Skłodowskiej-Curie,3,4,ul,ul,subst:sg:nom.acc:m3,['nazwa_pospolita'],[],ul,[ul]
172401,2019,WARSZAWA,Aktywny Trening Słuchowy Neuroflow,2,3,Słuchowy,słuchowy,adj:sg:nom.voc:m1.m2.m3:pos,[],[],słuchowy,[słuchowy]
137774,2019,KRAKÓW,Dzieci - super nauka i zabawa to jest nasza wspólna sprawa,2,3,super,super:A,adj:pl:nom.voc:m2.m3.f.n:pos,[],[],super,[super]
137265,2019,KRAKÓW,Wyskocz na rower,2,3,rower,rower,subst:sg:nom.acc:m3,['nazwa_pospolita'],[],rower,[rower]
50261,2019,TYCHY,Rodzinny festyn na terenie SP nr 18,1,2,festyn,festyn,subst:sg:nom.acc:m3,['nazwa_pospolita'],[],festyn,[festyn]
29531,2019,CZĘSTOCHOWA,"Doposazenie Ochotniczej Strazy Pozarnej Kuznica Marianowa, Liszka Dolna i Kawodrza Górna W sprzet oraz lekkie samochody ratowniczo-gasnicze",10,11,Kawodrza,Kawodrza,subst:sg:nom:f,['człon_nazwy_geograficznej'],[],Kawodrza,[kawodrza]
122150,2019,ŁÓDŹ,Rowerem wzdłuż Brzezińskiej - zapewnienie ciągłości drogi rowerowej.,6,7,drogi,drogi:A,adj:sg:nom.voc:m1.m2.m3:pos,[],[],drogi,[drogi]
25867,2019,BYTOM,"""Bezpiecznie i zdrowo do celu - stworzenie miasteczka ruchu drogowego W celach szkoleniowo-rekreacyjno - sportowych na terenie SP nr 36 przy ul. Siemiradzkiego 9. Wykonanie miasteczka oraz biezni na dzialce o wymiarach 50m X 35 m - Z nawierzchni utwardzonej, wyposazonego W ulice, chodniki, znaki, sygnalizacjq swietlna, wysepki, progi zwalniajace, skrzyzowania dróg, przejazdy dla rowerów, przej...",55,56,sygnalizacjq,sygnalizacjq,ign,[],[],sygnalizacjq,[sygnalizacjq]


In [58]:
#tfidf
processed_docs = df_2019_clean['tokens']
vocab = gensim.corpora.Dictionary(processed_docs)
corpus = [vocab.doc2bow(doc) for doc in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=5, id2word=vocab, passes=2, workers=10)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.060*"ul" + 0.054*"nowy" + 0.028*"do" + 0.022*"sportowy" + 0.019*"nawierzchnia" + 0.018*"nowa" + 0.017*"boisko" + 0.016*"parkingowy" + 0.013*"sp" + 0.013*"zajęcie"
Topic: 1 
Words: 0.054*"zielony" + 0.046*"plac" + 0.020*"zabawić" + 0.018*"podstawowy" + 0.018*"budowa" + 0.015*"centrum" + 0.012*"łódź" + 0.012*"open" + 0.009*"plenerowy" + 0.009*"warsztat"
Topic: 2 
Words: 0.085*"ulica" + 0.027*"mini" + 0.026*"budowo" + 0.020*"numer" + 0.017*"zieleń" + 0.014*"miejsce" + 0.013*"warszawa" + 0.011*"wraz" + 0.011*"budynek" + 0.008*"przestrzeń"
Topic: 3 
Words: 0.027*"szkoła" + 0.022*"osiedle" + 0.021*"chodnik" + 0.019*"dzielnica" + 0.016*"teren" + 0.013*"biblioteka" + 0.013*"on" + 0.012*"oraz" + 0.012*"mały" + 0.011*"droga"
Topic: 4 
Words: 0.044*"park" + 0.038*"dziecko" + 0.029*"zabawa" + 0.028*"pieszy" + 0.024*"miejski" + 0.021*"bezpieczny" + 0.021*"rowerowy" + 0.020*"stary" + 0.019*"młodzież" + 0.018*"nr"


In [59]:
# Wizualizacja tematów
vis_data = vis.prepare(lda_model, corpus_tfidf, vocab)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis_data)

## 2020

In [60]:
df_2020= pd.read_csv('/content/drive/My Drive/Konkursy Kaggle/Budżet obywatelski/Morfeusz_2020.csv')
df_2020.head()

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_
0,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",0,1,Doposażenie,doposażyć,ger:sg:nom.acc:n:perf:aff,[],[]
1,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",1,2,parku,park,subst:sg:gen:m3,['nazwa_pospolita'],[]
2,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",1,2,parku,park,subst:sg:loc:m3,['nazwa_pospolita'],[]
3,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",1,2,parku,park,subst:sg:voc:m3,['nazwa_pospolita'],[]
4,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",2,3,pomiędzy,pomiędzy,prep:acc,[],[]


In [61]:
df_2020['lemma_clean'] = df_2020['lemma'].str.split(':', expand=True)[0]
df_2020['tokens'] = df_2020['lemma_clean'].apply(simple_preprocess)
df_2020_ = df_2020[df_2020["tokens"].str.len() != 0]

In [62]:
#tfidf
processed_docs = df_2020_['tokens']
vocab = gensim.corpora.Dictionary(processed_docs)
corpus = [vocab.doc2bow(doc) for doc in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=5, id2word=vocab, passes=2, workers=10)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.234*"na" + 0.058*"dla" + 0.022*"dzielnica" + 0.021*"oraz" + 0.017*"numer" + 0.016*"teren" + 0.015*"centrum" + 0.013*"boisko" + 0.012*"mały" + 0.011*"droga"
Topic: 1 
Words: 0.081*"ulica" + 0.044*"przy" + 0.036*"łódź" + 0.029*"miejski" + 0.021*"chodnik" + 0.019*"rowerowy" + 0.019*"osiedle" + 0.017*"nr" + 0.015*"zabawić" + 0.012*"sp"
Topic: 2 
Words: 0.057*"ul" + 0.032*"pieszy" + 0.028*"mini" + 0.024*"budowo" + 0.023*"zabawa" + 0.021*"nowa" + 0.015*"zielone" + 0.015*"nawierzchnia" + 0.012*"miejsce" + 0.010*"parking"
Topic: 3 
Words: 0.048*"nowy" + 0.047*"do" + 0.036*"plac" + 0.017*"zieleń" + 0.016*"sportowy" + 0.016*"podstawowy" + 0.016*"stary" + 0.015*"budowa" + 0.013*"remonta" + 0.013*"biblioteka"
Topic: 4 
Words: 0.064*"zielony" + 0.038*"park" + 0.024*"szkoła" + 0.022*"dziecko" + 0.020*"bezpieczny" + 0.012*"młodzież" + 0.012*"remont" + 0.012*"on" + 0.008*"miasto" + 0.008*"zielona"


In [64]:
tags = ['prep', 'conj', 'interj', 'part']
df_2020_clean = df_2020_[~df_2020_['tag'].str.contains('|'.join(tags))]
df_2020_clean.sample(10)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_,lemma_clean,tokens
8991,2020,GDYNIA,"Uporządkowanie terenu jezdni ul. Krzywej, wjazd od ul. Kartuskiej, teren za sklepem Rossmann, utworzenie 4 miejsc parkingowych dla osób z niepełnoprawnością oraz montaż dwóch lamp LED",5,6,Krzywej,Krzywa,subst:sg:dat.loc:f,"['nazwa_geograficzna', 'nazwisko']",[],Krzywa,[krzywa]
47490,2020,KATOWICE,"Wykonanie jednego ogrodu deszczowego, W którym beda sadzone rosliny hydrofitowe. Prace beda prowadzone W zakresie przygotowania do sadzenia roslin oraz samego sadzenia.",3,4,deszczowego,deszczowy,adj:sg:gen:m1.m2.m3.n:pos,[],[],deszczowy,[deszczowy]
140016,2020,OPOLE,Ścieżki rowerowe,0,1,Ścieżki,Ścieżka:Sf,subst:pl:nom.acc.voc:f,['nazwisko'],[],Ścieżka,[ścieżka]
151544,2020,WARSZAWA,"Mniej plastiku na Białołęce - butelkomaty do plastiku''\r\n\t\tKulturalny butelkomat\r\n\t\tCiekawe spotkania z ciekawymi ludźmi - autorami książek, podróżnikami, reporterami, ilustratorami - w białołęckich bibliotekach\r\n\t\t20 drzew dla Białołęki\r\n\t\tEnglish book i german buch - czytamy w oryginale\r\n\t\tBezpieczny Zaułek - doświetlenie przystanku Zaułek Szkoła 02\r\n\t\tRemont boisk w ...",49,50,Remont,remonta,subst:pl:gen:f,['nazwa_pospolita'],['przest.'],remonta,[remonta]
133249,2020,ŁÓDŹ,ZAKUP NOWYCH KSIĄŻEK DLA UŻYTKOWNIKÓW BIBLIOTEKI PRZY WILCZEJ.,1,2,NOWYCH,Nowa,subst:pl:gen:f,"['nazwa_geograficzna', 'nazwisko']",[],Nowa,[nowa]
141490,2020,KROSNO,Zielony skwerek w Suchodole,0,1,Zielony,zielony:Sm1,subst:sg:nom:m1,['nazwa_pospolita'],[],zielony,[zielony]
170000,2020,WARSZAWA,"Dla Mnie, dla Ciebie, dla Nas – zintegrowany program wsparcia dla opiekunów dorosłych osób niepełnosprawnych, cierpiących z powodu demencji, zespołów otępiennych i choroby Alzheimera",18,19,cierpiących,cierpieć,pact:pl:gen.loc:m1.m2.m3.f.n:imperf:aff,[],[],cierpieć,[cierpieć]
109005,2020,GORZÓW WLKP.,Budowa 2 zatok parkingowych przy ul. Śląskiej 8-12,0,1,Budowa,Budowo,subst:pl:nom.acc.voc:n:ncol,['nazwa_geograficzna'],[],Budowo,[budowo]
49317,2020,KATOWICE,"W ramach rewitalizacji zostana przeprowadzone prace przywracajace terenowi walory estetyczne i rekreacyjno- uzyteczne , co bedzie miato ogromnie pozytywny wptyw nie tylko dla tego fragmentu dzielnicy Zatque, ale dla catego otoczenia, tj. Kolonii, lezacego W Tysiaclecia, DTS i centrum Zateza. To wtasnie ten teren od kilkudziesieciu lat jest codziennie obserwowany nie tylko przez mieszkanców Kol...",122,123,terenowych,terenowy,adj:pl:loc:m1.m2.m3.f.n:pos,[],[],terenowy,[terenowy]
23211,2020,GDAŃSK,Plac kalisteniczny (street workout) - rozbudowa infratruktury sportowo-rekreacyjnej przy Zespole Szkół Ogólnokształcących nr 6 w Gdańsku. Etap 1.,13,14,Zespole,zespół,subst:sg:voc:m3,['nazwa_pospolita'],[],zespół,[zespół]


In [65]:
#tfidf
processed_docs = df_2020_clean['tokens']
vocab = gensim.corpora.Dictionary(processed_docs)
corpus = [vocab.doc2bow(doc) for doc in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=5, id2word=vocab, passes=2, workers=10)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.052*"nowy" + 0.021*"nowa" + 0.020*"rowerowy" + 0.015*"remonta" + 0.010*"parkingowy" + 0.009*"łodzia" + 0.009*"oda" + 0.009*"oświetlenie" + 0.008*"wolny" + 0.008*"budynek"
Topic: 1 
Words: 0.082*"zielony" + 0.065*"ul" + 0.047*"park" + 0.036*"pieszy" + 0.031*"szkoła" + 0.020*"podstawowy" + 0.019*"zieleń" + 0.018*"sportowy" + 0.018*"nr" + 0.016*"centrum"
Topic: 2 
Words: 0.041*"plac" + 0.028*"mini" + 0.020*"dzielnica" + 0.019*"osiedle" + 0.018*"numer" + 0.017*"zielone" + 0.016*"teren" + 0.014*"remont" + 0.014*"biblioteka" + 0.014*"on"
Topic: 3 
Words: 0.091*"ulica" + 0.030*"miejski" + 0.030*"dziecko" + 0.028*"budowo" + 0.027*"zabawa" + 0.026*"bezpieczny" + 0.022*"chodnik" + 0.018*"nawierzchnia" + 0.017*"zabawić" + 0.013*"sp"
Topic: 4 
Words: 0.033*"łódź" + 0.023*"do" + 0.016*"budowa" + 0.015*"młodzież" + 0.015*"stary" + 0.014*"wola" + 0.011*"mały" + 0.010*"parking" + 0.010*"to" + 0.009*"nowość"


In [66]:
# Wizualizacja tematów
vis_data = vis.prepare(lda_model, corpus_tfidf, vocab)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis_data)

In [74]:
stopwords = ['do', 'wraz', 'oraz', 'ul', 'ulica', 'nr', 'numer', 'to', 'on', 'przy', 'dla', 'na', 'po','jak', 'ten', 'nie', 'od', 'oda']
df_2020_clean4 = df_2020_[~df_2020_['lemma'].str.contains('|'.join(stopwords))]
df_2020_clean4.sample(10)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_,lemma_clean,tokens
135626,2020,NOWY SĄCZ,"Integracja, sport i zabawa na Starym Mieście",6,7,Starym,Stara,subst:pl:dat:f,"['nazwa_geograficzna', 'nazwisko']",[],Stara,[stara]
77015,2020,POZNAŃ,Murem za chórem!,1,2,za,za:P,prep:gen,[],[],za,[za]
87076,2020,WROCŁAW,"ŻERNICZANIN - park aktywności z placem zabaw i siłownią, który „rośnie” razem z dzieckiem",10,11,który,który,adj:sg:acc:m3:pos,[],[],który,[który]
16253,2020,GDAŃSK,Bylinowa aleja stanowiąca uatrakcyjnienie ciągu pieszego od osiedla przy Kurpińskiego do budynków Zespołu Szkolno Przedszkolnego nr 2.,3,4,uatrakcyjnienie,uatrakcyjnić,ger:sg:nom.acc:n:perf:aff,[],[],uatrakcyjnić,[uatrakcyjnić]
72326,2020,KALISZ,Zieleniec – teren reprezentacyjno-wypoczynkowy przed głównym wejsciem do Szkoły Podstawowej nr 12.,6,7,przed,przed,prep:inst:nwok,[],[],przed,[przed]
157935,2020,WARSZAWA,Wiaty rowerowe przy Szkole podstawowej nr 10,1,2,rowerowe,rowerowy,adj:pl:acc:m2.m3.f.n:pos,[],[],rowerowy,[rowerowy]
77873,2020,POZNAŃ,Bezpieczne strefy rekreacji przy stawkach,2,3,rekreacji,rekreacja,subst:pl:gen:f,['nazwa_pospolita'],"['książk.,hom.']",rekreacja,[rekreacja]
38288,2020,KATOWICE,"Zakup nowości wydawniczych (książek i audiobooków), organizacja spotkań autorskich i prelekcji tematycznych oraz doposażenie czytelni",17,18,czytelni,czytelnia,subst:sg:gen:f,['nazwa_pospolita'],[],czytelnia,[czytelnia]
75293,2020,KONIN,ZIELONE PRZYSTANKI AUTOBUSOWE MZK wniosek,3,4,MZK,MZK,subst:sg.pl:nom.gen.dat.acc.inst.loc.voc:n:ncol,['nazwa_firmy'],[],MZK,[mzk]
168291,2020,WARSZAWA,Komfortowe nawierzchnie siłowni plenerowych i urządzeń street workout na Ursynowie,2,3,siłowni,siłownia,subst:sg:dat.loc:f,['nazwa_pospolita'],[],siłownia,[siłownia]


In [76]:
#tfidf
processed_docs = df_2020_clean4['tokens']
vocab = gensim.corpora.Dictionary(processed_docs)
corpus = [vocab.doc2bow(doc) for doc in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=7, id2word=vocab, passes=2, workers=10)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.072*"pieszy" + 0.055*"zabawa" + 0.045*"rowerowy" + 0.037*"zabawić" + 0.025*"drogi" + 0.024*"sp" + 0.017*"edukacyjny" + 0.014*"publiczny" + 0.013*"ogród" + 0.012*"nowe"
Topic: 1 
Words: 0.070*"plac" + 0.055*"miejski" + 0.040*"nowa" + 0.030*"centrum" + 0.021*"ścieżka" + 0.018*"nowość" + 0.015*"warsztaty" + 0.011*"letni" + 0.011*"wszyscy" + 0.011*"ruch"
Topic: 2 
Words: 0.090*"nowy" + 0.029*"stary" + 0.023*"miejsce" + 0.019*"parkingowy" + 0.016*"pies" + 0.016*"wolny" + 0.016*"plenerowy" + 0.015*"oświetlić" + 0.015*"bezpłatny" + 0.014*"warsztat"
Topic: 3 
Words: 0.060*"dziecko" + 0.042*"zieleń" + 0.026*"droga" + 0.025*"mały" + 0.019*"zakupić" + 0.018*"książka" + 0.015*"przejście" + 0.015*"wszystek" + 0.014*"open" + 0.013*"miejska"
Topic: 4 
Words: 0.035*"dzielnica" + 0.035*"osiedle" + 0.030*"boisko" + 0.020*"parking" + 0.017*"sala" + 0.015*"budynek" + 0.014*"filia" + 0.013*"aktywny" + 0.012*"czyli" + 0.011*"przedszkole"
Topic: 5 
Words: 0.083*"park" + 0.057*"szkoła" + 0.